In [1]:
import glob
import pandas as pd
from collections import Counter
import datetime as dt
import pyreadr
import warnings
warnings.filterwarnings("ignore")

regras_ignorados = {}
regras_ignorados['LOCNASC'] = [9]
regras_ignorados['ESTCIVMAE'] = [9]
regras_ignorados['ESCMAE'] = [9]
regras_ignorados['GESTACAO'] = [9]
regras_ignorados['GRAVIDEZ'] = [9]
regras_ignorados['PARTO'] = [9]
regras_ignorados['CONSULTAS'] = [9]
regras_ignorados['CONSULTAS'] = [9]
regras_ignorados['SEXO'] = [0, 9, 'I']
regras_ignorados['RACACOR'] = [9]
regras_ignorados['IDANOMAL'] = [8,9]
regras_ignorados['ESCMAE2010'] = [9]
regras_ignorados['TPMETESTIM'] = [8,9]
regras_ignorados['TPMETESTIM'] = [99]
regras_ignorados['TPAPRESENT'] = [9]
regras_ignorados['STTRABPART'] = [9]
regras_ignorados['STCESPARTO'] = [9]
regras_ignorados['TPNASCASSI'] = [9]
regras_ignorados['TPFUNCRESP'] = [0]
regras_ignorados['ESCMAEAGR1'] = [9]
regras_ignorados['TPROBSON'] = [11,12]
regras_ignorados['IDADEMAE'] = [99]
regras_ignorados['PESO'] = [9999]


In [2]:
#df = pyreadr.read_r('SINASC2021.rds') 
df = pd.read_csv('DN22OPEN_T.csv')
df1 = df.copy()
print(len(df))

2560320


In [33]:
#df1 = pd.DataFrame.from_dict(df[None])

In [3]:
ano = 2022
codmun = df1['CODMUNNASC']
df_ignorados = df1.copy()
df_totais = df1.isna()
df_nulos = df_totais.copy()
    
df_totais['ANO'] = ano
df_totais['CODMUNNASC'] = codmun

df_totais = df_totais.groupby(['ANO','CODMUNNASC']).count().reset_index().melt(id_vars=['ANO','CODMUNNASC'])

df_totais.columns = ['ANO','CODMUNNASC','VARIAVEL','TOTAIS']

df_nulos['CODMUNNASC'] = codmun
df_nulos['ANO'] = ano

df_nulos = df_nulos.groupby(['ANO','CODMUNNASC']).sum().reset_index().melt(id_vars=['ANO','CODMUNNASC'])
df_nulos.columns = ['ANO','CODMUNNASC','VARIAVEL','NULOS']

for c in df_ignorados.columns:
    if c in regras_ignorados:
        df_ignorados[c] = df_ignorados[c].isin(regras_ignorados[c])
    else:
        if c not in ['ANO','CODMUNNASC']:
            df_ignorados.drop(columns=[c], inplace=True)

df_ignorados['CODMUNNASC'] = codmun
df_ignorados['ANO'] = ano

df_ignorados = df_ignorados.groupby(['ANO','CODMUNNASC']).sum().reset_index().melt(id_vars=['ANO','CODMUNNASC'])
df_ignorados.columns = ['ANO','CODMUNNASC','VARIAVEL','IGNORADOS']

df_ignorados = df_ignorados.fillna(0)

x = df_totais.merge(df_nulos,how='left',on=['ANO','CODMUNNASC','VARIAVEL'])
x = x.merge(df_ignorados, how='left', on=['ANO','CODMUNNASC','VARIAVEL'])
x = x.reset_index()

x = x[['ANO','CODMUNNASC','VARIAVEL','NULOS','IGNORADOS','TOTAIS']]

x = x.fillna(0)  

In [4]:
x

,ANO,CODMUNNASC,VARIAVEL,NULOS,IGNORADOS,TOTAIS
0,2022,110001,contador,0,0.0,54
1,2022,110002,contador,0,0.0,2268
2,2022,110003,contador,0,0.0,1
3,2022,110004,contador,0,0.0,1784
4,2022,110005,contador,0,0.0,140
...,...,...,...,...,...,...
447059,2022,522190,res_coordenadas,0,0.0,1
447060,2022,522200,res_coordenadas,0,0.0,181
447061,2022,522205,res_coordenadas,0,0.0,68
447062,2022,522220,res_coordenadas,0,0.0,5


In [5]:
incompletude = x[~x.VARIAVEL.isin(['contador','NOVO'])]
incompletude.to_csv('SINASC_2022_incompletude.csv',index=None)
      

In [6]:
colunas_implausibilidade = ['ANO','ESTADO','CODMUNNASC','LOCNASC',
                        'IDADEMAE','ESTCIVMAE','ESCMAE','QTDFILVIVO',
                        'QTDFILMORT','GESTACAO','GRAVIDEZ','PARTO',
                        'CONSULTAS','DTNASC','HORANASC','SEXO',
                        'APGAR1','APGAR5','RACACOR','PESO','IDANOMAL',
                        'DTCADASTRO','CODANOMAL','ESCMAE2010','DTNASCMAE',
                        'QTDGESTANT','QTDPARTNOR','QTDPARTCES','IDADEPAI',
                        'DTULTMENST','SEMAGESTAC','TPMETESTIM','TPAPRESENT',
                        'STTRABPART','STCESPARTO','TPNASCASSI','TPFUNCRESP',
                        'TPDOCRESP','TPROBSON','SERIESCMAE','CONSPRENAT',
                        'MESPRENAT','ESCMAEAGR1','PARIDADE']

# aplica as regras para variaveis com opcoes
regras_gerais = { 'LOCNASC':[1,2,3,4,5,9],
                'ESTCIVMAE':[1,2,3,4,5,9],
                'ESCMAE':[1,2,3,4,5,9],
                'GESTACAO':[1,2,3,4,5,6,9],
                'GRAVIDEZ':[1,2,3,9],
                'PARTO':[1,2,9],
                'CONSULTAS':[1,2,3,4,9],
                'SEXO':[1,2,9,0,'M','F','I'],
                'RACACOR':[1,2,3,4,5],
                'IDANOMAL':[1,2,9],
                'ESCMAE2010':[1,2,3,4,5,9],
                'TPMETESTIM':[1,2,9],
                'TPAPRESENT':[1,2,3,9],
                'STTRABPART':[1,2,3,9],
                'STCESPARTO':[1,2,3,9],
                'TPNASCASSI':[1,2,3,4,9],
                'TPFUNCRESP':[1,2,3,4,5,9],
                'TPDOCRESP':[1,2,3,4,5],
                'TPROBSON': list(range(1,13)), # 1 a 12
                'SERIESCMAE': list(range(1,9)), # 1 a 8
                'MESPRENAT': list(range(1,11)) + [99], # 1 a 10 e 99
                'ESCMAEAGR1': list(range(1,13)), # 1 a 12,
            }

# incompletude ###################################################



ano = 2021
df = df1
aux_cols = []
for c in colunas_implausibilidade:
    if c in df1.columns:
        aux_cols.append(c)

aux = df1[aux_cols]

aux['ANO'] = ano
aux['CODMUNNASC'] = codmun


for col in regras_gerais.keys():
    if col in aux_cols:
        aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & (~aux[col].isin(regras_gerais[col]))
    
# REGRAS ESPECÍFICAS

col = 'IDADEMAE'
if col in aux_cols:
    aux[col] = pd.to_numeric(aux[col], errors='coerce')

    aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & ((aux[col] < 10) | (aux[col] > 55))

for col in ['QTDFILVIVO','QTDFILMORT']:
    if col not in aux_cols:
        continue
    aux[col] = pd.to_numeric(aux[col], errors='coerce')
    aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & ((aux[col] < 0) | (aux[col] > 70))
    
col = 'PESO'
if col in aux_cols:
    aux[col] = pd.to_numeric(aux[col], errors='coerce')
    aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & ((aux[col] < 0) | (aux[col] > 11000))

for col in ['QTDGESTANT','QTDPARTNOR','QTDPARTCES']:
    if col not in aux_cols:
        continue
    aux[col] = pd.to_numeric(aux[col], errors='coerce')
    aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & ((aux[col] < 0) | (aux[col] > 27))

col = 'IDADEPAI'
if col in aux_cols:
    aux[col] = pd.to_numeric(aux[col], errors='coerce')
    aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & ((aux[col] < 10) | (aux[col] > 99))

col = 'SEMAGESTAC'
if col in aux_cols:
    aux[col] = pd.to_numeric(aux[col], errors='coerce')
    aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & (aux[col] < 20)
    
col = 'CONSPRENAT'
if col in aux_cols:
    aux[col] = pd.to_numeric(aux[col], errors='coerce')
    aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & (aux[col] < 0)
    
for col in ['DTNASC','DTCADASTRO']:
    if col in aux_cols:
        aux[col] = pd.to_numeric(aux[col].astype(str).str[-4:],errors='coerce')
        aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & (aux[col] > dt.date.today().year)
    
col = 'DTNASCMAE'
if col in aux_cols:
    aux[col] = pd.to_numeric(aux[col].astype(str).str[-4:],errors='coerce')
    aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & ((aux[col] > 2012) | (aux[col] < 1967))
    
col = 'HORANASC'
if col in aux_cols:
    if df[col].dtype == "object":
        df[col] = df[col].str.replace(";","")
        df[col] = pd.to_numeric(df[col], errors='coerce')
    hora = pd.to_numeric(df[col], errors='coerce') // 100
    minuto = pd.to_numeric(df[col], errors='coerce') % 100
    aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & (df[col] > 59) & (hora > 23) & (minuto > 59) # 00:59 vira 59 só
    
for col in ['APGAR1','APGAR5']:
    if col in aux_cols:
        aux[col] = pd.to_numeric(aux[col], errors='coerce')
        aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & ((aux[col] < 0) | (aux[col] > 10))
    
col = 'PARIDADE'
if col in aux_cols:
    aux[col] = pd.to_numeric(aux[col], errors='coerce')
    aux[col + "_IMPLAUSIVEL"] = (~aux[col].isna()) & ((aux[col] < 0) | (aux[col] > 27))

aux_cols = []

for c in aux.columns:
    if 'IMPLAUSIVEL' in c:
        aux_cols.append(c)
        
aux_cols = ['ANO','CODMUNNASC'] + aux_cols

df_implausiveis = aux[aux_cols]

df_implausiveis.fillna(0, inplace=True)

df_implausiveis = df_implausiveis.groupby(['ANO','CODMUNNASC']).sum().reset_index().melt(id_vars=['ANO','CODMUNNASC'])
df_implausiveis.columns = ['ANO','CODMUNNASC','VARIAVEL','IMPLAUSIVEIS']    


df['ANO'] = ano
df['CODMUNNASC'] = codmun

df_totais = df[['ANO','CODMUNNASC']]
df_totais['TOTAIS'] = 1

df_totais = df_totais.groupby(['ANO','CODMUNNASC'])['TOTAIS'].sum().reset_index()
df_totais.columns = ['ANO','CODMUNNASC','TOTAIS']
    
df_totais.set_index(['ANO','CODMUNNASC'], inplace=True)
df_implausiveis.set_index(['ANO','CODMUNNASC'], inplace=True)
    
x = df_totais.join([df_implausiveis], how='left')
x = x.reset_index()

x = x[['ANO','CODMUNNASC','VARIAVEL','IMPLAUSIVEIS','TOTAIS']]

x = x.fillna(0)  



In [8]:
x.to_csv('SINASC_Implausibilidade_2022.csv', index=None)

In [9]:


regras = {}
regras['LOCNASC_e_PARTO'] = "Se campo LOCNASC for 2,3,4,5 e o campo PARTO estiver preenchido com 2"
regras['PARTO_e_STCESPARTO'] = "Se o campo STCESPARTO estiver preenchido como 1 e o campo PARTO estiver como 2 ou 9"
regras['TPROBSON_e_composicao'] = "Se o campo TPROBSON estiver preenchido entre 1 e 10 e qualquer um dos campos QTDGESTANT,QTDPARTNOR, QTDPARTCES,SEMAGESTAC,TPAPRESENT, STTRABPART estiverem em branco"                                          
# inconsistencia


df = df1

ano = 2021
codmun = df['CODMUNNASC']

df['PESO'] = df['PESO'].apply(pd.to_numeric, errors='coerce')

df['parto_prematuro'] = df['GESTACAO'].apply(pd.to_numeric, errors='coerce') <= 4

aux_cols = []

base = df

# LOCNASC e PARTO
base['LOCNASC_e_PARTO_INCONSISTENTES'] = (base['LOCNASC'].isin([2,3,4,5])) & (base['PARTO'] == 2)

# PARTO e STCESPARTO
if 'STCESPARTO' in base.columns:
        base['PARTO_e_STCESPARTO_INCONSISTENTES']=(base['STCESPARTO'] == 1) & (base['PARTO'].isin([2,9]))

# TPROBSON e composicao
if 'TPROBSON' in base.columns:
        base['TPROBSON_e_composicao_INCONSISTENTES'] = (base['TPROBSON'].isin([1,2,3,4,5,6,7,8,9,10])) & ((~base[['QTDGESTANT','QTDPARTNOR','QTDPARTCES','SEMAGESTAC','TPAPRESENT','STTRABPART']].isna()).sum(axis = 1) > 0)

# PARTO_PREMATURO e PESO
base['PARTO_PREMATURO_e_PESO_INCONSISTENTES']=(base['parto_prematuro'] == 1) & (base['PESO'] > 2500)

aux_cols = []
for c in base.columns:
        if 'INCONSISTENTES' in c:
                aux_cols.append(c)
        
aux = base[aux_cols]

aux['ANO'] = ano
aux['CODMUNNASC'] = codmun

df_inconsistentes = aux

df_inconsistentes.fillna(0, inplace=True)

df_inconsistentes = df_inconsistentes.groupby(['ANO','CODMUNNASC']).sum().reset_index().melt(id_vars=['ANO','CODMUNNASC'])
df_inconsistentes.columns=['ANO','CODMUNNASC','VARIAVEL','INCONSISTENTES']

df['ANO'] = ano
df['CODMUNNASC'] = codmun

df_totais = df[['ANO','CODMUNNASC']]
df_totais['TOTAIS'] = 1

df_totais = df_totais.groupby(['ANO','CODMUNNASC'])['TOTAIS'] \
                        .sum().reset_index()
df_totais.columns = ['ANO','CODMUNNASC','TOTAIS']

df_totais.set_index(['ANO','CODMUNNASC'], inplace=True)
df_inconsistentes.set_index(['ANO','CODMUNNASC'], inplace=True)

x = df_totais.join([df_inconsistentes], how='left')
x = x.reset_index()

x = x[['ANO','CODMUNNASC','VARIAVEL','INCONSISTENTES','TOTAIS']]

x = x.fillna(0)  


In [10]:
x

,ANO,CODMUNNASC,VARIAVEL,INCONSISTENTES,TOTAIS
0,2021,110001,LOCNASC_e_PARTO_INCONSISTENTES,0,54
1,2021,110001,PARTO_e_STCESPARTO_INCONSISTENTES,24,54
2,2021,110001,TPROBSON_e_composicao_INCONSISTENTES,54,54
3,2021,110001,PARTO_PREMATURO_e_PESO_INCONSISTENTES,2,54
4,2021,110002,LOCNASC_e_PARTO_INCONSISTENTES,0,2268
...,...,...,...,...,...
15411,2021,522220,PARTO_PREMATURO_e_PESO_INCONSISTENTES,0,5
15412,2021,530010,LOCNASC_e_PARTO_INCONSISTENTES,8,48763
15413,2021,530010,PARTO_e_STCESPARTO_INCONSISTENTES,12840,48763
15414,2021,530010,TPROBSON_e_composicao_INCONSISTENTES,48402,48763


In [11]:
x.to_csv('SINASC_Inconsistencia_2022.csv',index=None)